In [5]:
import math as m
import pandas as pn
import numpy as np
import random


### ACO

In [6]:

class ACO(object):
    def __init__(self, ant_count: int, generations: int, alpha: float, beta: float, rho: float, q: int,
                 strategy: int):
        """
        :param ant_count:
        :param generations:
        :param alpha: relative importance of pheromone
        :param beta: relative importance of heuristic information
        :param rho: pheromone residual coefficient
        :param q: pheromone intensity
        :param strategy: pheromone update strategy. 0 - ant-cycle, 1 - ant-quality, 2 - ant-density
        
        """
        self.Q = q
        self.rho = rho
        self.beta = beta
        self.alpha = alpha
        self.ant_count = ant_count
        self.generations = generations
        self.update_strategy = strategy

    def _update_pheromone(self, graph: Graph, ants: list):
        
        for i, row in enumerate(graph.pheromone):
            for j, col in enumerate(row):
                
                graph.pheromone[i][j] *= self.rho
                
                for ant in ants:
                    graph.pheromone[i][j] += ant.pheromone_delta[i][j]
                    

    
    def solve(self, graph: Graph):
        """
        :param graph:
        
        """
        best_cost = float('-inf')
        best_solution = []
        
        for gen in range(self.generations):
            
            ants = []
            ant = _Ant(self, graph, random.randint(0, graph.rank - 1))
            ants.append(ant)
            
            
            for loop in range(self.ant_count):
                for i in range(graph.rank - 1):
                    ant._select_next()
                
                
                '''
                ########### Testing #####################
                
                length = (len(ant.tabu))
                summ = 0

                for i in range (1, length):
                    summ += graph.matrix[ant.tabu[i - 1]][ant.tabu[i]]

                
                print (str(ant.total_cost) + " VS " + str(summ))
               
               ############################################
                
                '''
                
                
                if ant.total_cost > best_cost:
                    best_cost = ant.total_cost
                    best_solution = [] + ant.tabu
                    
                    
                # update pheromone
                ant._update_pheromone_delta()
                ant._remember_best_path(best_solution)
                
                if loop != (self.ant_count - 1):
                    ant = _Ant(self, graph,  random.randint(0, graph.rank - 1))
                    ants.append(ant)
            
            
            self._update_pheromone(graph, ants)
            
        return best_solution, best_cost




### _Helper Classes_

In [7]:
class Graph(object):
    def __init__(self, cost_matrix: list, rank: int):
        """
        :param cost_matrix:
        :param rank: rank of the cost matrix
        
        """
        self.matrix = cost_matrix
        self.rank = rank
        self.pheromone = [[1/rank for j in range(rank)] for i in range(rank)]
        
        
class _Ant(object):
    
    def __init__(self, aco: ACO, graph: Graph, start):
        
        self.colony = aco
        self.graph = graph
        self.total_cost = 0.0
        
        # tabu = The path travelled by the ant - list
        self.tabu = [] 

        # pheromone_delta = the local increase of pheromone
        self.pheromone_delta = []  
        
        # allowed = nodes which are allowed for the next selection - already visited are excluded
        self.allowed = [i for i in range(graph.rank)]  
        
        #eta =  heuristic information
        self.eta = [[0 if i == j else (graph.matrix[i][j] * 0.1) for j in range(graph.rank)] for i in range(graph.rank)]  
        
        #start node
        self.start = start  
        
        #Start node insertion
        self.tabu.append(start)
        self.current = start
        self.allowed.remove(start)

        
        
    def _select_next(self):
        denominator = 0   # for normalization purpose
        
        
        """
        The costs (self.graph.matrix[][]) are added with probability to add weight to the nodes with more cost

        """
        for i in self.allowed:
            denominator += self.graph.pheromone[self.current][i] ** self.colony.alpha * self.eta[self.current][i] \
                                                    ** self.colony.beta * self.graph.matrix[self.current][i] ** 3
            
        
        # probabilities for moving to a node in the next step
        probabilities = [0 for i in range(self.graph.rank)]  
        
        for i in range(self.graph.rank):
            
            try:
                self.allowed.index(i)  # test if allowed list contains i
                
                
                """
                The costs (self.graph.matrix[][]) are added with probability to add weight to the nodes with more cost
                
                """
                probabilities[i] = self.graph.pheromone[self.current][i] ** self.colony.alpha * \
                    self.eta[self.current][i] ** self.colony.beta * self.graph.matrix[self.current][i] ** 3 / denominator
            
            
            except ValueError:
                pass  # do nothing
            
            
        # select next node by probability roulette. 
        selected = 0
        
        rand = random.random()
        for i, probability in enumerate(probabilities):
            rand -= probability
            
            if rand <= 0:
                selected = i
                break
        
        
        self.allowed.remove(selected)
        self.tabu.append(selected)
        self.total_cost += self.graph.matrix[self.current][selected]
        self.current = selected
        
    
    
    def _update_pheromone_delta(self):
        
        self.pheromone_delta = [[0 for j in range(self.graph.rank)] for i in range(self.graph.rank)]
        
        for _ in range(1, len(self.tabu)):
            i = self.tabu[_ - 1]
            j = self.tabu[_]
            
            '''
            ant-density system -

            Modified : self.pheromone_delta[i][j] = self.colony.Q * self.graph.matrix[i][j] ** 2
            Before modification : self.pheromone_delta[i][j] = self.colony.Q / self.graph.matrix[i][j]
            
            '''
            
            if self.colony.update_strategy == 1:  # ant-quality system
                self.pheromone_delta[i][j] = self.colony.Q
                
            elif self.colony.update_strategy == 2:  # ant density System
                self.pheromone_delta[i][j] = self.colony.Q * self.graph.matrix[i][j] ** 2
            
            else:  # ant-cycle system
                self.pheromone_delta[i][j] = self.colony.Q / self.total_cost
                
    
    
    """
    The fuction is added so that the best path found by an ant has more pheromone. 
    The next ants can then recognize the better parts by tracing the paths with more pheromone
    
    """
    def _remember_best_path(self, best_path_till_now):
        
        for _ in range (1, len(best_path_till_now)):
            i = best_path_till_now[_ - 1]
            j = best_path_till_now[_]
            
            #graph.matrix[][] is multiplied so that the nodes with more weight has more pheromone
            self.pheromone_delta[i][j] += self.colony.Q * (self.graph.matrix[i][j] ** (1))   
        
        



### DNA Fragment Assembly Problem using ACO

In [12]:


def main():
    
    cnt = 0
    cost_matrix = []
    
    with open('./data/x60189_4.csv') as f:
        
        for line in f.readlines():
            cnt = cnt + 1
            if (cnt > 2):
                row = line.split(',')
                n = len(row)
                
                row [n - 1] = row [n - 1].replace("\n", "")
                row = list(map(int, row))

                cost_matrix.append(row)
                
        
    
    
    rank = (len(cost_matrix))
   
    ant_number = 1000
    genarations = 5
    alpha = 10
    beta = 0
    rho = 0.5          
    q = 0.1            
    strategy = 2       # pheromone update strategy 
    
    aco = ACO(ant_number, genarations, alpha, beta, rho, q, strategy)
    
    
    graph = Graph(cost_matrix, rank)
    path, cost = aco.solve(graph)
    
    tot=0
    for i in range(len(path)-1):
        tot=tot+cost_matrix[path[i]][path[i+1]]
    print(tot)

    
    print('Cost: {}\nPath: {}'.format(cost, path))
    
    
    
    return path, cost_matrix



In [13]:
#Maximum
import time

start = time.time()
path = []
cost_matrix = []

#if __name__ == '__main__':
path, cost_matrix = main()
    
end = time.time()

print ("Time taken: " + str(end - start))





11499
Cost: 11499.0
Path: [9, 33, 0, 3, 10, 25, 5, 12, 18, 16, 15, 13, 27, 20, 14, 6, 26, 22, 17, 4, 37, 31, 2, 23, 34, 11, 28, 29, 38, 24, 19, 36, 30, 21, 1, 7, 8, 35, 32]
Time taken: 42.55827069282532


In [11]:
tot=0
for i in range(len(path)-1):
    tot=tot+cost_matrix[path[i]][path[i+1]]
print(tot)




11499


Testing

In [ ]:
path1 = [9, 33, 0, 3, 10, 25, 5, 12, 18, 16, 15, 13, 27, 20, 14, 6, 26, 22, 17, 4, 37, 31, 2, 23, 34, 11, 28, 29, 38, 24, 19, 36, 30, 21, 1, 7, 8, 35, 32]

tot=0
for i in range(len(path1)-1):
    tot=tot+cost_matrix[path1[i]][path1[i+1]]
print(tot)

